# Raw Data Processing

## DBLP paper ciation data loading

In [2]:
file_path = '/Users/zongchang/Desktop/实验数据集/DBLP-Paper'

In [4]:
with open(file_path + '/DBLPOnlyCitationOct19.txt', 'r') as f:
    paper_list = []
    paper = {}
    paper_raw = []
    for line in f.readlines():
        if line == '\n':
            ref_index = []
            for item in paper_raw:
                if item.startswith('#*'):
                    paper['title'] = item.replace('#*', '')
                elif item.startswith('#@'):
                    paper['author'] = item.replace('#@', '')
                elif item.startswith('#t'):
                    paper['year'] = item.replace('#t', '')
                elif item.startswith('#c'):
                    paper['venue'] = item.replace('#c', '')
                elif item.startswith('#index'):
                    paper['index'] = item.replace('#index', '')
                elif item.startswith('#%'):
                    ref_index.append(item.replace('#%', ''))
                elif item.startswith('#!'):
                    paper['abstract'] = item.replace('#!', '')
                else:
                    continue
            paper['ref_index'] = ref_index
            paper_list.append(paper)
            paper = {}
            paper_raw = []
        else:
            paper_raw.append(line.strip())

In [9]:
# filter paper without abstract, references and year earlier than 2000
paper_list_filter = []
for paper in paper_list:
    if ('abstract' not in paper) or (int(paper['year']) < 2000) or (not paper['ref_index']):
        continue
    else:
        paper_list_filter.append(paper)

In [10]:
len(paper_list_filter)

223995

## DBLP paper citation count (indicator_input.pkl)

In [18]:
# index yearly citation dict for papers at year 2000
index_cite_count = {}
for paper in paper_list_filter:
    if paper['year'] == '2000':
        index_cite_count[paper['index']] = {}

# count citation
for paper in paper_list_filter:
    for ref in paper['ref_index']:
        if ref in index_cite_count:
            if paper['year'] not in index_cite_count[ref]:
                index_cite_count[ref][paper['year']] = 1
            else:
                index_cite_count[ref][paper['year']] += 1

        

In [19]:
# keep papers have citations
index_cite_count_nonzero = {}
for k,v in index_cite_count.items():
    if v:
        index_cite_count_nonzero[k] = v

In [31]:
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
citation_count_list = [] # index, year, citation_count
for k,v in index_cite_count_nonzero.items():
    for year in years:
        if year not in v:
            tmp = [k, year, 0]
        else:
            tmp = [k, year, v[year]]
        citation_count_list.append(tmp)

In [43]:
len(list(index_cite_count_nonzero.keys()))

4791

In [45]:
import pickle
with open('./data/dblp_paper/paper_4791_citation.pkl', 'wb') as f:
    pickle.dump(citation_count_list, f)

In [37]:
idx_2_paper_index = {}
idx = 0
for paper_index in list(index_cite_count_nonzero.keys()):
    idx_2_paper_index[idx] = paper_index
    idx += 1

In [39]:
indicator_citation = []
for idx in range(len(list(idx_2_paper_index.keys()))):
    tmp = []
    for indicator in citation_count_list:
        if indicator[0] == idx_2_paper_index[idx]:
            tmp.append([[indicator[2]], indicator[1]])
    
    indicator_citation.append(tmp)

In [40]:
indicator_citation

[[[[0], '2000'],
  [[1], '2001'],
  [[3], '2002'],
  [[0], '2003'],
  [[2], '2004'],
  [[2], '2005'],
  [[6], '2006'],
  [[3], '2007'],
  [[5], '2008'],
  [[3], '2009'],
  [[4], '2010']],
 [[[0], '2000'],
  [[2], '2001'],
  [[2], '2002'],
  [[2], '2003'],
  [[2], '2004'],
  [[3], '2005'],
  [[1], '2006'],
  [[2], '2007'],
  [[3], '2008'],
  [[1], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[1], '2002'],
  [[0], '2003'],
  [[1], '2004'],
  [[0], '2005'],
  [[1], '2006'],
  [[1], '2007'],
  [[1], '2008'],
  [[0], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[1], '2001'],
  [[3], '2002'],
  [[3], '2003'],
  [[3], '2004'],
  [[3], '2005'],
  [[4], '2006'],
  [[1], '2007'],
  [[5], '2008'],
  [[2], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[0], '2002'],
  [[0], '2003'],
  [[0], '2004'],
  [[1], '2005'],
  [[0], '2006'],
  [[0], '2007'],
  [[0], '2008'],
  [[0], '2009'],
  [[0], '2010']],
 [[[0], '2000'],
  [[0], '2001'],
  [[5], '2002'],
  [[4], '

In [44]:
import pickle
with open('./data/dblp_paper/indicator_year_input.pkl', 'wb') as f:
    pickle.dump(indicator_citation, f)

## DBLP paper graph (graph_input.pkl)